In [ ]:
import csv
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np

In [ ]:
#nltk.download("stopwords")
import psycopg2
import psycopg2.extras
import os

conn = psycopg2.connect(host='localhost', user='user', password='hidden', dbname='postgres')

sql = """
SELECT subreddit FROM subreddit_subset_small 
"""
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(sql)
subreddit_subset = cur.fetchall()
cur.close()
conn.close()

In [ ]:
def cleanDoc(doc):
    stopset = set(stopwords.words('english'))
    stemmer = nltk.PorterStemmer()
    doc = re.sub(r'\W',' ',doc)
    tokens = WordPunctTokenizer().tokenize(doc)
    clean = [token.lower() for token in tokens if token.lower() not in stopset and len(token) > 2]
    final = [stemmer.stem(word) for word in clean]
    return final

In [ ]:
#subreddits = pickle.load(open('post_titles_clean.p','rb'))
sub_subset_list = []
for item in subreddit_subset:
    sub_subset_list.append(item[0])


In [ ]:
print len(subreddit_subset)

In [ ]:
subreddits = {}

with open('post_titles.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')

    for row in reader:
        subreddit, title = row
        if subreddit in subreddit_subset:
            if subreddit not in subreddits.keys():
                cleanTitle = cleanDoc(title)
                #print(subreddit, title, cleanTitle)
                subreddits[subreddit] = cleanTitle#re.sub(r'\W',' ',title)
            else:
                cleanTitle = cleanDoc(title)
                #print('plus',subreddit, title, cleanTitle, subreddits[subreddit])
                templist = subreddits[subreddit]
                #print templist
                templist.extend(cleanTitle)
                subreddits[subreddit] = templist #re.sub(r'\W',' ',title)

In [ ]:
subreddits = pickle.load(open('post_titles_clean.p','rb'))

In [ ]:
post_titles = []
for sub, titles in subreddits.iteritems():
    if sub in sub_subset_list:
        #print sub
        titles_str = ' '.join(titles)
        post_titles.append(titles_str)
        
#with open('post_titles_combined.csv', 'wb') as wfile:
#    writer = csv.writer(wfile)
#    writer.writerows(post_titles)

In [ ]:
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True)#, stop_words = 'english')
sklearn_representation = sklearn_tfidf.fit_transform(post_titles)

In [ ]:
pairwise_similarity = sklearn_representation * sklearn_representation.T

In [ ]:
data = pairwise_similarity.data # similarity
indices = pairwise_similarity.indices # sub2
indptr = pairwise_similarity.indptr

In [ ]:
subs_indices = []
for i, ptr in enumerate(indptr):
    if i != len(indptr):
        nextptr = indptr[i+1]
    #print i, ptr, nextptr
    for j in range(ptr,nextptr):
        subs_indices.append(i)
len(subs_indices)

In [ ]:
print len(data)
print len(subs_indices)

In [ ]:
csv_filename = 'post_titles_subset_pairwise-small.csv'
with open(csv_filename, 'wb') as wfile:
    writer = csv.writer(wfile)
    for i in range(len(data)): 
        row = [subs_indices[i],indices[i],data[i]]
        writer.writerow(row)

In [ ]:
#to get actual subreddit names use number from sub.csv, match with subreddits.keys()
#to get actual words, use index match with sklearn_representation.get_feature_names()
#np.savetxt('post_titles_subs.csv', thing)
feature_names = sklearn_tfidf.get_feature_names()
subs = subreddits.keys()

with open('post_titles_subset_sklearn_representation.p', 'wb') as wfile:
    pickle.dump(sklearn_representation,wfile)

with open('post_titles_subset_features_keys', 'wb') as wfile:
    writer = csv.writer(wfile)
    writer.writerows(feature_names)

with open('post_titles_subset_sub_names', 'wb') as wfile:
    writer = csv.writer(wfile)
    writer.writerows(subs)
